Data Sources: 

In [79]:
import pandas as pd
from sqlalchemy import create_engine, inspect

### Store CSV into DataFrame

In [80]:
worldometer_csv = "Resources/worldometer_data.csv"
worldometer_df = pd.read_csv(worldometer_csv)
worldometer_df.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa


In [81]:
usa_csv = "Resources/usa_county_wise.csv"
usa_df = pd.read_csv(usa_csv)
usa_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0,0


### Create new data with select columns

In [82]:
worldometer_df = worldometer_df[['Country/Region', 'Population', 'TotalCases', 'TotalDeaths', 'Tot Cases/1M pop', 'Tot Cases/1M pop', 'TotalTests', 'Tests/1M pop']].copy()
worldometer_df.head()

,Country/Region,Population,TotalCases,TotalDeaths,Tot Cases/1M pop,Tot Cases/1M pop,TotalTests,Tests/1M pop
0,USA,3.311981e+08,5032179,162804.0,15194.0,15194.0,63139605.0,190640.0
1,Brazil,2.127107e+08,2917562,98644.0,13716.0,13716.0,13206188.0,62085.0
2,India,1.381345e+09,2025409,41638.0,1466.0,1466.0,22149351.0,16035.0
3,Russia,1.459409e+08,871894,14606.0,5974.0,5974.0,29716907.0,203623.0
4,South Africa,5.938157e+07,538184,9604.0,9063.0,9063.0,3149807.0,53044.0


In [83]:
usa_df = usa_df[['Admin2', 'Province_State', 'Date', 'Confirmed', 'Deaths']].copy()
usa_df.head()

,Admin2,Province_State,Date,Confirmed,Deaths
0,NaN,American Samoa,1/22/20,0,0
1,NaN,Guam,1/22/20,0,0
2,NaN,Northern Mariana Islands,1/22/20,0,0
3,Adjuntas,Puerto Rico,1/22/20,0,0
4,Aguada,Puerto Rico,1/22/20,0,0


In [84]:
usa_df = usa_df.dropna()
usa_df


,Admin2,Province_State,Date,Confirmed,Deaths
3,Adjuntas,Puerto Rico,1/22/20,0,0
4,Aguada,Puerto Rico,1/22/20,0,0
5,Aguadilla,Puerto Rico,1/22/20,0,0
6,Aguas Buenas,Puerto Rico,1/22/20,0,0
7,Aibonito,Puerto Rico,1/22/20,0,0
...,...,...,...,...,...
627915,Central Utah,Utah,7/27/20,347,1
627916,Southeast Utah,Utah,7/27/20,70,0
627917,Southwest Utah,Utah,7/27/20,2781,23
627918,TriCounty,Utah,7/27/20,142,0


In [85]:
usa_df = usa_df.rename(columns={"Admin2": "County", "Province_State": "Province/State", "Confirmed": "Cases"})
usa_df





,County,Province/State,Date,Cases,Deaths
3,Adjuntas,Puerto Rico,1/22/20,0,0
4,Aguada,Puerto Rico,1/22/20,0,0
5,Aguadilla,Puerto Rico,1/22/20,0,0
6,Aguas Buenas,Puerto Rico,1/22/20,0,0
7,Aibonito,Puerto Rico,1/22/20,0,0
...,...,...,...,...,...
627915,Central Utah,Utah,7/27/20,347,1
627916,Southeast Utah,Utah,7/27/20,70,0
627917,Southwest Utah,Utah,7/27/20,2781,23
627918,TriCounty,Utah,7/27/20,142,0


### Connect to local database

In [86]:
connection_string = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{connection_string}')

### Check for tables

In [87]:
inspector = inspect(engine)

In [88]:
inspector.get_table_names()

['world_covid_stats', 'usa_covid_stats']

### Use pandas to load csv converted DataFrame into database

In [89]:
usa_df.to_sql(name='usa_covid_stats', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [91]:
pd.read_sql_query('select * from world_covid_stats', con=engine).head()

,Country/Region,Population,TotalCases,TotalDeaths,Tot Cases/1M pop,TotalTests,Tests/1M pop
0,USA,3.311981e+08,5032179,162804.0,15194.0,63139605.0,190640.0
1,Brazil,2.127107e+08,2917562,98644.0,13716.0,13206188.0,62085.0
2,India,1.381345e+09,2025409,41638.0,1466.0,22149351.0,16035.0
3,Russia,1.459409e+08,871894,14606.0,5974.0,29716907.0,203623.0
4,South Africa,5.938157e+07,538184,9604.0,9063.0,3149807.0,53044.0


In [93]:
pd.read_sql_query('select * from usa_covid_stats', con=engine).head()

,County,Province/State,Date,Cases,Deaths
0,Adjuntas,Puerto Rico,1/22/20,0,0
1,Aguada,Puerto Rico,1/22/20,0,0
2,Aguadilla,Puerto Rico,1/22/20,0,0
3,Aguas Buenas,Puerto Rico,1/22/20,0,0
4,Aibonito,Puerto Rico,1/22/20,0,0
